In [1]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Reading in each csv file for the imputed data
data = pd.read_csv('imputed_data.csv', encoding = 'ISO-8859-1')
data.head()

,Unnamed: 0,year,agent,state,zip_code,lp_amount,lp_type,prod,status,income,months,18_months,ACH,credit_card,paper_check
0,0,2014.0,Church Bulletin 0011 (109398),MA,1001.0,34.95,Refund,1.0,Returned <30 days,74467.0,0.87,0,0,1,0
1,1,2015.0,Benefits and More 4 Paid (143595),MA,1001.0,29.95,Sale,1.0,Returned < 90 days,74467.0,0.00,0,0,1,0
2,2,2016.0,Benefits and More 4 Paid (143595),MA,1001.0,29.95,Sale,1.0,Switched to LW,74467.0,0.00,0,0,1,0
3,3,2016.0,Benefits and More 4 Paid (143595),MA,1001.0,39.95,Refund,1.0,Returned <30 days,74467.0,0.67,0,0,1,0
4,5,2014.0,Newspaper (109455),MA,1002.0,34.95,Sale,1.0,Decline Cancel,55937.0,0.57,0,0,1,0
